## Import the necessary libraries

In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import math
import scipy
import scipy.interpolate
import random
import pylab as plb
import scipy.optimize as opt
from loop_analysis import *
from sklearn.linear_model import LinearRegression
import time
from numpy import diff
from scipy.signal import find_peaks
from scipy.signal import argrelextrema
from scipy.optimize import curve_fit, least_squares
from scipy.stats import norm
from scipy.optimize import curve_fit

## Plot averaged loops

In [3]:
pi = 45
direction = 'x'
file_name = '5.h5'
signal_type = 'woll1_diff'
############################################# Load file data ###################################################################
folder_name = r'C:\Users\Balbo\OneDrive\Desktop\Uni\MSciProject\James\Si\The Real Deal'
#folder_name = r'C:\Users\migels\Miguel\PhD\MOKE\Measures\NdCo520nm\B'+str(direction)
#folder_name = r'C:\Users\user\Desktop\Miguel\Measurements\SP3621\B'+str(direction)
filepath = os.path.join(folder_name, file_name)
data = get_averaged_loop(filepath,1000)
# Correct the difference and sum of signals from the detectors
data['woll1_diff'] = data['woll1_det1'] - data['woll1_det2']
data['woll1_sum'] = data['woll1_det1'] + data['woll1_det2']
## Title construction
# What we are measuring: rotation or ellipticity
if file_name[0] == 'e': 
    mtype = 'ellipticity'
else:
    mtype = 'rotation'
# How we are using our prism
if file_name[2] == str(4):
    if file_name[4] == 'm':
        wollo = 'wollaston at 45 manually'        
    else:
        wollo = 'wollaston at 45 based on detectors'
else:
    wollo = 'wollaston aligned with s and p'   
  
incidence_angle = (180 - 2*pi)/2
full_title = 'NdCo5 ' + str(mtype) + ' with ' + str(wollo) + ' at ' + str(incidence_angle)+ ' deg'  
################################################################################################################################
############################################### Here we choose what signal to use ##############################################
cutoff1 = -20
cutoff2 = 20
################################################################################################################################
######################################### Extract the data and manipulate ######################################################
# Get magnetic field on sample reference frame
y = data[signal_type]
y,a = straighten_loop(data,cutoff1,cutoff2,direction,signal_type)
#y,a = straighten_loop(data,cutoff1,cutoff2,direction,signal_type)
bxt = np.array(data['fields_X'])
byt = np.array(data['fields_Y'])
bzt = np.array(data['fields_Z'])
Bxsample = []
#bxt = np.append(bxt,0)
#dx = diff(bxt)
#y = data[signal_type]
for i in range(len(bxt)):
    aux = math.sqrt(bxt[i]**2 + bzt[i]**2)
    if bxt[i] < 0:
        Bxsample.append(-aux)
    else:
        Bxsample.append(aux)

Bzsample = []
for i in range(len(bzt)):
    aux = math.sqrt(bxt[i]**2 + bzt[i]**2)
    if bzt[i] < 0:
        Bzsample.append(-aux)
    else:
        Bzsample.append(aux)

# Create figure
%matplotlib qt
fig=plt.figure(figsize=(9, 5), dpi= 200, facecolor='w', edgecolor='k')
plt.rcParams.update({'font.size': 16})
if direction=='x': plt.plot(bxt, y, linewidth = 0.3)
if direction=='y': plt.plot(byt, y, linewidth = 3)
if direction=='z': plt.plot(Bzsample, y, linewidth = 3)
plt.grid()
plt.xlabel('B' + str(direction) + ' [mT]')
if signal_type == 'woll1_diff': tipo = ' difference '
if signal_type == 'woll1_sum': tipo = ' sum '
if signal_type == 'woll1_det1': tipo = ' detector 1 '
if signal_type == 'woll1_det2': tipo = ' detector 2 '  
plt.gca().invert_xaxis()
plt.ylabel('MOKE' + tipo + 'signal')
#plt.xlim(-20,20)
#plt.title(full_title)
plt.tight_layout()
saving_name = os.path.splitext(filepath)[0]+'_'+signal_type+'.png'
plt.savefig(saving_name)

## Differential Plots

In [7]:
pi = 45
############################################# Load file data ###################################################################
#folder_name = r'C:\Users\Balbo\OneDrive\Desktop\Uni\MSciProject\James\Kapton\85K'
#folder_name = r'C:\Users\migels\Miguel\PhD\MOKE\Measures\NdCo520nm\B'+str(direction)
#folder_name = r'C:\Users\user\Desktop\Miguel\Measurements\SP3621\B'+str(direction)
filepath = os.path.join(folder_name, file_name)
data = get_averaged_loop(filepath,1000)
# Correct the difference and sum of signals from the detectors
data['woll1_diff'] = data['woll1_det1'] - data['woll1_det2']
data['woll1_sum'] = data['woll1_det1'] + data['woll1_det2']
## Title construction
# What we are measuring: rotation or ellipticity
if file_name[0] == 'e': 
    mtype = 'ellipticity'
else:
    mtype = 'rotation'
# How we are using our prism
if file_name[2] == str(4):
    if file_name[4] == 'm':
        wollo = 'wollaston at 45 manually'        
    else:
        wollo = 'wollaston at 45 based on detectors'
else:
    wollo = 'wollaston aligned with s and p'   
  
incidence_angle = (180 - 2*pi)/2
full_title = 'NdCo5 ' + str(mtype) + ' with ' + str(wollo) + ' at ' + str(incidence_angle)+ ' deg'  
################################################################################################################################
############################################### Here we choose what signal to use ##############################################
cutoff1 = -20
cutoff2 = 20
################################################################################################################################
######################################### Extract the data and manipulate ######################################################
# Get magnetic field on sample reference frame

#y,a = straighten_loop(data,cutoff1,cutoff2,direction,signal_type)
bxt = np.array(data['fields_X'])
byt = np.array(data['fields_Y'])
bzt = np.array(data['fields_Z'])
Bxsample = []



#bxt = np.append(bxt,0)

def smooth(a,WSZ):
    # a: NumPy 1-D array containing the data to be smoothed
    # WSZ: smoothing window size needs, which must be odd number,
    # as in the original MATLAB implementation
    out0 = np.convolve(a,np.ones(WSZ,dtype=int),'valid')/WSZ    
    r = np.arange(1,WSZ-1,2)
    start = np.cumsum(a[:WSZ-1])[::2]/r
    stop = (np.cumsum(a[:-WSZ:-1])[::2]/r)[::-1]
    return np.concatenate((  start , out0, stop  ))

limx = 30.0
#y = data[signal_type]
y = smooth(y,5)
y = y[bxt > -limx]
bxt = bxt[bxt > -limx]
y = y[bxt < limx]
bxt = bxt[bxt < limx]
dx = diff(bxt)
dy = diff(y)
y = -dy/dx
y = np.append(y,0)

#for i in range(len(bxt)):
    #aux = math.sqrt(bxt[i]**2 + bzt[i]**2)
    #if bxt[i] < 0:
        #Bxsample.append(-aux)
    #else:
        #Bxsample.append(aux)

#Bzsample = []
#for i in range(len(bzt)):
    #aux = math.sqrt(bxt[i]**2 + bzt[i]**2)
    #if bzt[i] < 0:
        #Bzsample.append(-aux)
    #else:
        #Bzsample.append(aux)

# Create figure
%matplotlib qt
fig=plt.figure(figsize=(9, 5), dpi= 200, facecolor='w', edgecolor='k')
plt.rcParams.update({'font.size': 16})
if direction=='x': plt.plot(bxt, y, linewidth = 0.3)
if direction=='y': plt.plot(byt, y, linewidth = 3)
if direction=='z': plt.plot(Bzsample, y, linewidth = 3)
plt.grid()
plt.xlabel('B' + str(direction) + ' [mT]')
if signal_type == 'woll1_diff': tipo = ' difference '
if signal_type == 'woll1_sum': tipo = ' sum '
if signal_type == 'woll1_det1': tipo = ' detector 1 '
if signal_type == 'woll1_det2': tipo = ' detector 2 '  

#plt.gca().invert_yaxis()
plt.ylabel('dMOKE' + tipo + 'signal/dB')
#plt.xlim(-30,30)
#plt.ylim(0,0.1*max(y) + max(y))
plt.ylim(0,0.004)
#plt.title(full_title)
#locs, labels = xticks()
plt.xticks(fontsize=7, rotation=90)
plt.xticks(np.arange(-limx, limx, 1))
plt.tight_layout()
saving_name = os.path.splitext(filepath)[0]+'d_'+signal_type+'.png'
plt.savefig(saving_name)

IndexError: boolean index did not match indexed array along dimension 0; dimension is 557 but corresponding boolean dimension is 999

## Parameters

In [107]:
x = bxt

def gauss(x, a, mu, sig):
    return a*np.exp(-(x-mu)**2/(2*sig**2))


lim1 = 2
lim2 = 14

fitx=x[(x>lim1)*(x<lim2)]
fity=y[(x>lim1)*(x<lim2)]
mu=np.sum(fitx*fity)/np.sum(fity)
sig=np.sqrt(np.sum(fity*(fitx-mu)**2)/np.sum(fity))


interp_func = scipy.interpolate.interp1d(y, bxt)
maxx = max(fity)
Hc1 = interp_func(maxx) 
print('Hc1 = ', Hc1)
print('sigma1 = ', sig)
print('FWHM = ', sig*2.355)
sigavg = sig


lim3 = -15
lim4 = -3

fitx2=x[(x>lim3)*(x<lim4)]
fity2=y[(x>lim3)*(x<lim4)]
mu=np.sum(fitx2*fity2)/np.sum(fity2)
sig=np.sqrt(np.sum(fity2*(fitx2-mu)**2)/np.sum(fity2))

maxx = max(fity2)
Hc2 = interp_func(maxx) 
print('Hc2 = ', Hc2)
print('sigma2 = ', sig)
print('FWHM = ', sig*2.355)
Hc = ((Hc1 - Hc2)/2)
print('<Hc> = ', Hc)
sigavg = (sigavg + sig)/2
print('<sigma> = ',sigavg)
Hb = ((Hc1 + Hc2)/2)
print ('Hb = ', Hb)

Hc1 =  5.407546999330523
sigma1 =  2.1660135553650637
FWHM =  5.100961922884725
Hc2 =  -5.121167351303565
sigma2 =  2.2329260448859842
FWHM =  5.2585408357064924
<Hc> =  5.264357175317044
<sigma> =  2.1994698001255237
Hb =  0.14318982401347924


## Sigma

In [122]:
# Effective code
print(maxx)
#difference = maxx - min(y)
difference = maxx - 0
HM = difference / 2
print(HM)
pos_extremum = np.where(y == maxx)  # or in your case: arr_y.argmin()
pos_extremum = pos_extremum[0]
pos_extremum = pos_extremum[0]
print(pos_extremum)
nearest_above = (np.abs(y[pos_extremum:-1] - HM)).argmin()
nearest_below = (np.abs(y[0:pos_extremum] - HM)).argmin()

FWHM = (np.mean(bxt[nearest_above + pos_extremum]) - 
        np.mean(bxt[nearest_below]))  
print(FWHM)
sigma = FWHM/2.355 
print(sigma)
print(y[42])

0.003937035339638888
0.001968517669819444
42
-18.94569193632556
-8.044879803110641
0.003937035339638888


## Mapping

In [166]:
pos = np.arange(25)
Hc = np.random.rand(25)
fig=plt.figure(figsize=(9, 5), dpi= 200, facecolor='w', edgecolor='k')
a_dataframe = pd.DataFrame({"column1": [1, 2, 3, 4, 5], "column1": [6, 7 , 8, 9, 10], "column1": [11, 12, 13, 14, 15], "column1": [16, 17, 18, 19, 20],"column1": [21, 22, 23, 24, 25]})
plt.pcolor(a_dataframe)
plt.yticks(np.arange(0.5, len(a_dataframe.index), 1), a_dataframe.index)
plt.xticks(np.arange(0.5, len(a_dataframe.columns), 1), a_dataframe.columns)
saving_name = os.path.splitext(filepath)[0]+'d_'+signal_type+'.png'
#plt.show()

pos = list(range(1, 26))
Hcval = random.sample(range(1, 100), 25)

Yrows = [1, 2, 3, 4, 5]
Xcol = [1, 2, 3, 4, 5,]

position = ((np.asarray(pos)).reshape(5,5))
Hcvalues = ((np.asarray(Hcval)).reshape(5,5))
print(position)
print(Hcvalues)

path_to_csv = r"C:\Users\Balbo\OneDrive\Desktop\Uni\MSciProject\Python"
df = pd.read_excel(path_to_csv ,index_col=0)

plt.imshow(df,cmap='hot',interpolation='nearest')

plt.show() 

labels = (np.asarray(["{0} \n {1:.2f}".format(symb,value) for symb, value in zip(position.flatten(), Hcvalues.flatten())])).reshape(5,5)

fig, ax = plt.subplots(figsize = (12,7))
title = "Map"

plt.title(title, fontsize = 18)
ttl = ax.title
ttl.set_position([0.5,1.05])

ax.set_xticks([])
ax.set_yticks([])
ax.axis('off')

#sns.heatmap(result,annot=labels,fmt="",cmap="RdYlGn",linewidths=0.30,ax=ax)
#plt.show()

H1cArray = []

[[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]
 [16 17 18 19 20]
 [21 22 23 24 25]]
[[65 15  7 85 50]
 [ 2 28 77 68 61]
 [55 53 27 83 52]
 [ 8 33 16 54 13]
 [59 89 62 81  4]]


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\Balbo\\OneDrive\\Desktop\\Uni\\MSciProject\\Python'

In [130]:
#fig=plt.figure(figsize=(9, 5), dpi= 200, facecolor='w', edgecolor='k')
data = np.genfromtxt("test.csv", delimiter=",")
plt.imshow(data, cmap='hot', interpolation='nearest')
plt.show()



In [8]:
file_name = 'sigma.csv'
folder_name = r'C:\Users\Balbo\OneDrive\Desktop\Uni\MSciProject\Python\K84'
filepath = os.path.join(folder_name, file_name)
data = np.genfromtxt(filepath, delimiter=",")

file_name1 = file_name.replace('.csv', '')
n_clusters = 10

heatmap = plt.pcolor(data)
figure = plt.colorbar(heatmap)
#figure.lines[0].set_linewidth(10)
#plt.colorbar(heatmap, ticks=np.arange(0,1,1./n),)
#plt.clim(-2.5,2.5)
#plt.imshow(data, cmap='hot', interpolation='nearest')
#ax = sns.heatmap(data, cbar_kws={'ticks': [0.0, 0.5, 1.0, 1.5, 2.0]}, vmin=0, vmax=2) 
##tick_locs = (np.arange(n_clusters) + 0.5)*(n_clusters-1)/n_clusters
#figure.set_ticks(tick_locs)
#figure.set_ticklabels(np.arange(n_clusters))
plt.title(file_name1, fontsize =20)
plt.tight_layout()
plt.show()
figure = heatmap.get_figure() 
saving_name = os.path.splitext(filepath)[0]+'.png'
figure.savefig(saving_name)

## Parameters graph

In [7]:
#file_name = 'sigma.csv'
#folder_name = r'C:\Users\Balbo\OneDrive\Desktop\Uni\MSciProject\Python'
filepath = r'C:\Users\Balbo\OneDrive\Desktop\Uni\MSciProject\Python'


Hc1Si83 = 3.164
Hc2Si83 = -0.860
HcSi83 = 2.012
HbSi83 = 1.15192
sigma1Si83 = 0.2698
sigma2Si83 = 0.2686
sigmaSi83 = 0.26924


Hc1K83 = 6.20524
#Hc1K84 = 6.94605
Hc1K84 = 6.796869565
Hc1K85 = 5.55575
sigma1K83 = 1.1278
sigma1K84 = 1.18915
sigma1K85 = 1.1012


Hc2K83 = -6.86688
#Hc2K84 = -6.53715
Hc2K84 = -6.351652174
Hc2K85 = -5.28215
sigma2K83 = 1.46176
sigma2K84 = 1.12685
sigma2K85 = 0.89405
HbK83 = -0.33082
HbK84 = 0.20445
HbK85 = 0.1368


x = [1.4, 1.6, 1.8]
y = [Hc1K83, Hc1K84, Hc1K85]
z = [Hc2K83, Hc2K84, Hc2K85]
w = [HbK83, HbK84, HbK85]
u = 0

xerr1 = [sigma1Si83]
xerr2 = [sigma2Si83]
xerr = [sigmaSi83]
yerr = [sigma1K83, sigma1K84, sigma1K85]
zerr = [sigma2K83, sigma2K84, sigma2K85]
werr = [(sigma1K83 + sigma2K83)/2, (sigma1K84 + sigma2K84)/2,(sigma1K85 + sigma2K85)/2]

fig, ax = plt.subplots()


ax.plot(1.4,Hc1Si83, '-bo')
ax.plot(1.4,Hc2Si83, '-bo',label='_nolegend_')
ax.plot(1.4,HbSi83, '-bo', label='_nolegend_')
ax.plot(x,y, '-ro')

#ax.annotate('local max', xy=(2, 1), xytext=(3, 1.5), arrowprops=dict(facecolor='black', shrink=0.05),)
#ax.annotate(xy=(x[-1],y[-1]), xytext=(5,0), textcoords='offset points', text='Hc1', va='center')
ax.plot(x,z, '-ro')
ax.plot(x,w, '-ro')

plt.errorbar(1.4,Hc1Si83, xerr1, fmt='bo')
plt.errorbar(1.4,Hc2Si83, xerr2, fmt='bo')
plt.errorbar(1.4,HbSi83, xerr, fmt='bo')


plt.errorbar(x, y, yerr, fmt='ro')
plt.errorbar(x, z, zerr, fmt='ro')
plt.errorbar(x, w, werr, fmt='ro')
#plt.errorbar(x, y, yerr, marker='s', mfc='red', mec='green', ms=20, mew=4)

ax.set_title('Bx vs Thickness')
ax.legend(['Si','Kapton'])
#leg = ax.get_legend()
#legendHandles[1].set_color('green')
ax.xaxis.set_label_text('thickness (nm)')
ax.yaxis.set_label_text('Bx (mT)')

plt.annotate('Hc1', xy=(1.8, 7.5))
plt.annotate('Hc2', xy=(1.8, -7.5))
plt.annotate('Hb', xy=(1.8, 1))

plt.annotate('Hc1', xy=(1.4, 3.5))
plt.annotate('Hc2', xy=(1.4, -0.5))
plt.annotate('Hb', xy=(1.4, 1.5))

plt.show()
saving_name = os.path.splitext(filepath)[0]+'.png'
plt.savefig(saving_name)



## Demagnetization

### Live

In [4]:
file_name = '1.h5'
mtype = 'Samp'
signal_type = 'woll1_diff'
direction = 'x'
OAMPS = np.array([3,1.3,1.08,1.03,1.01,0.99])
############################################# Load file data ###################################################################
folder_name = r'C:\Users\Balbo\OneDrive\Desktop\Uni\MSciProject\James\Kapton\85K'
filepath = os.path.join(folder_name, file_name)
data = get_averaged_loop(filepath,1000)
# Correct the difference and sum of signals from the detectors
data['woll1_diff'] = data['woll1_det1'] - data['woll1_det2']
data['woll1_sum'] = data['woll1_det1'] + data['woll1_det2']

################################################################################################################################
############################################### Here we choose what signal to use ##############################################
cutoff1 = -2
cutoff2 = 6
################################################################################################################################
######################################### Extract the data and manipulate ######################################################
# Get magnetic field on sample reference frame
y = data[signal_type]
y,a = straighten_loop(data,cutoff1,cutoff2,direction,signal_type)

bxt = np.array(data['fields_X'])
byt = np.array(data['fields_Y'])
bzt = np.array(data['fields_Z'])
Bxsample = []
for i in range(len(bxt)):
    aux = math.sqrt(bxt[i]**2 + bzt[i]**2)
    if bxt[i] < 0:
        Bxsample.append(-aux)
    else:
        Bxsample.append(aux)

# Create figure
%matplotlib qt
fig=plt.figure(figsize=(9, 5), dpi= 200, facecolor='w', edgecolor='k')
plt.rcParams.update({'font.size': 16})
scalingxaxis = 1.05
scalingyaxis = 1.05
yma = max(y)*scalingyaxis
ymi = min(y)*scalingyaxis
xma = max(Bxsample)*scalingxaxis
xmi = min(Bxsample)*scalingxaxis
plt.xlim(xmi,xma)
plt.ylim(ymi,yma)
plt.title('Loop progress in time')
plt.xlabel('Bx [mT]')
plt.ylabel('MOKE differential signal')
plt.grid()
for i in range(len(Bxsample)):
    if i >=1: plt.scatter(Bxsample[i-1], y[i-1], linewidth = 0.1, color = 'blue')
    plt.scatter(Bxsample[i], y[i], linewidth = 0.1, color = 'red')
    plt.pause(0.000001)

plt.show()


KeyboardInterrupt: 

### Averaged

In [4]:
file_name = '5.h5'
signal_type = 'woll1_diff'
direction = 'x'
############################################# Load file data ###################################################################
folder_name = r'C:\Users\Balbo\OneDrive\Desktop\Uni\MSciProject\James\Kapton\85K'
filepath = os.path.join(folder_name, file_name)
data = get_averaged_loop(filepath,1000)
# Correct the difference and sum of signals from the detectors
data['woll1_diff'] = data['woll1_det1'] - data['woll1_det2']
data['woll1_sum'] = data['woll1_det1'] + data['woll1_det2']


y = data[signal_type]
bxt = np.array(data['fields_X'])
byt = np.array(data['fields_Y'])
bzt = np.array(data['fields_Z'])
Bxsample = []
for i in range(len(bxt)):
    aux = math.sqrt(bxt[i]**2 + bzt[i]**2)
    if bxt[i] < 0:
        Bxsample.append(-aux)
    else:
        Bxsample.append(aux)

%matplotlib qt
fig, ax = plt.subplots(figsize=(9, 5), dpi= 200, facecolor='w', edgecolor='k')
plt.rcParams.update({'font.size': 16})
if direction=='x':
    plt.scatter(Bxsample, y, linewidth = 3)
else:
    plt.scatter(byt, y, linewidth = 3)
plt.grid()
plt.xlabel('Bx [mT]')
plt.ylabel('MOKE differential signal')
plt.tight_layout()
#saving_name = os.path.splitext(filepath)[0]+'.png'
#plt.savefig(saving_name)

## B field analysis

In [6]:
# Get the folder and file
pi = 30
folder_name =r'C:\Users\Balbo\OneDrive\Desktop\Uni\MSciProject\James\Kapton\85K' 
file_name = '5.h5'

# Extract the data and manipulate
filepath = os.path.join(folder_name, file_name)
data = get_averaged_loop(filepath,1000)
x = np.array(data['fields_X'])
z = np.array(data['fields_Z'])
ang = []
for i in range(len(x)):
        ang.append(abs(math.degrees(math.atan(z[i]/x[i]))))

# Here we plot the magnetic field in the three directions of the table reference frame and the angle that the field makes with x
# of the table
fig, ax1 = plt.subplots(figsize=(16, 15))
ax2 = ax1.twinx()
ax1.grid()
l1, = ax1.plot(data['t'], data['fields_X'], 'b-')
l1.set_label('Bx')
l2, = ax1.plot(data['t'], data['fields_Y'], 'g-')
l2.set_label('By')
l3, = ax1.plot(data['t'], data['fields_Z'], 'r-')
l3.set_label('Bz')
ax2.plot(data['t'],ang,color='purple')
ax1.legend()
plt.title('Study of magnetic fields (in table reference frame)')
ax1.set_xlabel('t [s]')
ax2.set_ylabel('Pi angle [°]', color='purple')
ax1.set_ylabel('B [mT]')

fig2=plt.figure(figsize=(6, 5), dpi= 200, facecolor='w', edgecolor='k')

plt.grid()
plt.plot(x,z)
plt.title('Study of magnetic fields (in table reference frame)')
plt.xlabel('Bx [mT]')
plt.ylabel('By [mT]')
plt.tight_layout()
saving_name = 'magfield.png'
plt.savefig(saving_name)

## Plot single loop

To plot a certain magnetic field direction we need the following input:
    Bx --> direction = 1
    By --> direction = 2
    Bz --> direction = 3

In [ ]:
direction = 1
loop_number = 15
direction2 = 1
loop_number2 = 2

In [ ]:
w1dif,w2dif,hall = plot_single_loop(filepath, direction, loop_number)
fig=plt.figure(figsize=(6, 3), dpi= 300, facecolor='w', edgecolor='k')
plt.rcParams.update({'font.size': 16})
plt.plot(hall,w1dif,linewidth=0.3)
plt.xlabel('B [mT]')
plt.ylabel('MOKE signal')
plt.tight_layout()
plt.show()

## Magnetic field study

In [3]:
hall = get_magnetic_field(filepath, direction, loop_number)
hall2 = get_magnetic_field(filepath, direction2, loop_number2)
fig=plt.figure(figsize=(6, 3), dpi= 300, facecolor='w', edgecolor='k')
plt.rcParams.update({'font.size': 16})
plt.plot(hall,linewidth=0.3)
plt.plot(hall2,linewidth=0.3,color='red')
plt.xlabel('Samples')
plt.ylabel('B [mT]')
plt.tight_layout()
plt.show()

NameError: name 'loop_number' is not defined